In [1]:
! pip install langchain langchain_pinecone openai unstructured langchain-text-splitters


In [2]:
! pip install -U langchain-community


In [3]:
! pip install langchain langchain_pinecone openai unstructured langchain-text-splitters "unstructured[pdf]" -U langchain-community


In [4]:
! pip install tiktoken


In [5]:
! pip install tiktoken -q


In [6]:
! pip install sentence-transformers


In [7]:
! pip install -U langchain-huggingface


In [8]:
! pip install torch


In [9]:
! pip install torch torchvision torchaudio


In [12]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [11]:
! pip install --upgrade torch


In [ ]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from pinecone import Pinecone as PineconeClient, ServerlessSpec


In [34]:
! pip install torch torchvision torchaudio

Loading Pdf Documents

In [45]:
! pip install torch pymupdf


In [43]:
! pip install torch


In [48]:
directory = 'C:/Users/HP/OneDrive/Desktop/RAG-ChatBot/datas'
def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
print(f"Number of documents loaded: {len(documents)}")

Error loading file C:\Users\HP\OneDrive\Desktop\RAG-ChatBot\datas\pdf12.pdf


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

Split documents into smaller chunks

In [47]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(f"Number of document chunks: {len(docs)}")

NameError: name 'documents' is not defined

In [8]:
docs[1]

Document(metadata={'source': '/content/datas/pdf_11.pdf'}, page_content='Supervised machine learning is the search for algorithms that reason from externally supplied instances to produce general hypotheses, which then make predictions about future instances. In other words, the goal of supervised learning is to build a concise model of the distribution of class labels in terms of predictor features. The resulting classifier is then used to assign class labels to the testing instances where the values of the predictor features are known, but the value of the class label is unknown. This paper describes various supervised machine learning classification techniques. Of course, a single article cannot be a complete review of all supervised machine learning classification algorithms (also known induction classification algorithms), yet we hope that the references cited will cover the major theoretical issues, guiding the researcher in interesting research directions and suggesting possible

Embeddings

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

Pinecone

In [11]:
from google.colab import userdata
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [12]:
# Initialize Pinecone
pc = PineconeClient(api_key="cdf36dac-4732-4ac9-ace0-d0f9d30d4d18")

# Index name
index_name = 'rag-chatbot'

# Check if 'rag-chatbot' exists and create if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Change the dimension to 384 to match the embeddings
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to the index
index = LangchainPinecone.from_documents(docs, embeddings, index_name=index_name)

print(f"Index '{index_name}' is set up and ready.")


Index 'rag-chatbot' is set up and ready.


In [13]:
def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = 'what is machine learning'
similar_docs =get_similiar_docs(query)
similar_docs

[Document(metadata={'source': '/content/datas/pdf12.pdf'}, page_content='Sebastian Raschka Department of Statistics University of Wisconsin–Madison\n\nhttp://stat.wisc.edu/∼sraschka/teaching/stat479-fs2018/\n\nFall 2018\n\n1 L01: What is Machine Learning? An Overview.\n\n1.1 Machine Learning – The Big Picture\n\nOne of the main motivations why we develop (computer) programs to automate various kinds of processes. Originally developed as a subﬁeld of Artiﬁcial Intelligence (AI), one of the goals behind machine learning was to replace the need for developing computer programs “manually.” Considering that programs are being developed to automate processes, we can think of machine learning as the process of “automating automation.” In other words, machine learning lets computers “create” programs (often, the intent for developing these programs is making predictions) themselves. In other words, machine learning is the process of turning data into programs.')]